In [17]:
import datetime
import operator
import os
import sys
import time
import pyspark
from operator import add
import numpy as np
import matplotlib.path as mplPath
from math import radians, cos, sin, asin, sqrt
start = time.time()

In [18]:
def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        #print type(zones.geometry[idx])
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1

In [19]:
def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones('block-groups-polygons-simple.geojson')
    for line in parts:
        if line.startswith('vendor_id'): continue 
        fields = line.strip('').split(',')
        if fields ==['']: continue
        if all((fields[5],fields[6],fields[9],fields[10])) and float(fields[4])<=2:
            passenger_count = int(fields[3])
            pickup_location  = geom.Point(proj(float(fields[5]), float(fields[6])))
            dropoff_location = geom.Point(proj(float(fields[9]), float(fields[10])))
            pickup_zone = findBlock(pickup_location, index, zones)
            dropoff_zone = findBlock(dropoff_location, index, zones)
            if pickup_zone>=0 and dropoff_zone>=0:
                yield ((dropoff_zone,pickup_zone), 1)

def mapper2(k2v2):
    from heapq import nlargest
    k, values = k2v2
    top3 = nlargest(3, values,key=lambda a: a[1])
    return (k,top3)

In [20]:
trips = sc.textFile('/home/satya/BDM_dataset/yellow_tripdata_2011-05.csv')
#output = trips.mapPartitions(mapToZone).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey().map(mapper2)
output = sc.parallelize(mapToZone(trips.take(2000)))
output1 = output.map(lambda x: (x[0][1],x[1])).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: -x[1])
output2 = output.map(lambda x: (x[0][0],x[1])).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: -x[1])
#output = sc.parallelize(output).reduceByKey(add).map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey().map(mapper2)
print output1
print output2
print (time.time()-start)/60.0

[(9245, 18), (8986, 15), (9144, 13), (9493, 13), (9509, 13), (9088, 11), (9365, 11), (9394, 10), (9052, 10), (9085, 10)]
[(9144, 26), (9245, 22), (9052, 15), (8986, 13), (9493, 12), (9610, 10), (9023, 10), (9594, 9), (9620, 8), (9509, 8)]
0.162078515689
